In [1]:
def gamma_correction(image, gamma = 0.8):
    invGamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** invGamma * 255 for i in range(256)]).astype("uint8")
    return cv2.LUT(image, table)

In [2]:
import os
import cv2
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

# Path to dataset
base_path = '//media//sparsh//CaptainSlow//Programming Stuff//Code _n_ Stuff//All Projects//All Projects//X-Ray Classifier//xray_iamge_dataset/'
train_folder = os.path.join(base_path, 'train')
test_folder = os.path.join(base_path, 'test')

# Define image transformations using torchvision
transform = transforms.Compose([
    transforms.ToPILImage(),  # Convert to PIL Image (needed for torchvision transforms)
    transforms.Grayscale(num_output_channels=1),  # Ensure 1-channel grayscale
    transforms.Resize((150, 150)),  # Resize
    transforms.Lambda(lambda img: cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8)).apply(np.array(img))),  # CLAHE
    transforms.Lambda(lambda img: cv2.GaussianBlur(img, (5, 5), 0)),  # Gaussian Blur
    transforms.ToTensor(),  # Convert to tensor & normalize to [0,1]
    transforms.Lambda(lambda img: img.expand(3, -1, -1)),  # Convert 1-channel to 3-channel
    transforms.Resize((224, 224)),  # Resize for ResNet
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet Normalization
])

# Custom Dataset Class
class XRayDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.image_paths = []
        self.labels = []

        # Load image paths and labels
        for label in os.listdir(folder_path):
            label_path = os.path.join(folder_path, label)
            for image_name in os.listdir(label_path):
                self.image_paths.append(os.path.join(label_path, image_name))
                self.labels.append(label)

        # Convert labels to integers
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(sorted(set(self.labels)))}
        self.labels = [self.class_to_idx[label] for label in self.labels]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        # Load image using OpenCV
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            raise RuntimeError(f"Error loading image: {img_path}")

        # Apply transformations
        if self.transform:
            img = self.transform(img)

        return img, torch.tensor(label, dtype=torch.long)

# Create dataset instances
train_dataset = XRayDataset(train_folder, transform=transform)
test_dataset = XRayDataset(test_folder, transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

print(f"Training Dataset: {len(train_dataset)} images")
print(f"Test Dataset: {len(test_dataset)} images")


Training Dataset: 5216 images
Test Dataset: 624 images


In [3]:
import torch
import torch.nn as nn
from torchvision import models


model = models.resnet50(pretrained=True)


num_classes = 2
model.fc = nn.Linear(model.fc.in_features, num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


/home/sparsh/miniconda3/envs/torch/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/sparsh/miniconda3/envs/torch/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
import torch.optim as optim

criterionn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

epochs = 100

for i in range(epochs):
    model.train()

    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterionn(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{i+1}/{epochs}], Loss: {running_loss:.4f}")

Epoch [1/100], Loss: 14.8968
Epoch [2/100], Loss: 3.8772
Epoch [3/100], Loss: 2.2513
Epoch [4/100], Loss: 1.8172
Epoch [5/100], Loss: 2.9471
Epoch [6/100], Loss: 2.0589
Epoch [7/100], Loss: 0.8873
Epoch [8/100], Loss: 2.5430
Epoch [9/100], Loss: 1.0001
Epoch [10/100], Loss: 1.0604
Epoch [11/100], Loss: 0.1940
Epoch [12/100], Loss: 0.0312
Epoch [13/100], Loss: 0.0641
Epoch [14/100], Loss: 0.0320
Epoch [15/100], Loss: 0.0163
Epoch [16/100], Loss: 0.0085
Epoch [17/100], Loss: 0.0059
Epoch [18/100], Loss: 0.0042
Epoch [19/100], Loss: 0.0042
Epoch [20/100], Loss: 0.0033
Epoch [21/100], Loss: 0.0031
Epoch [22/100], Loss: 0.0031
Epoch [23/100], Loss: 0.0025
Epoch [24/100], Loss: 0.0039
Epoch [25/100], Loss: 0.0034
Epoch [26/100], Loss: 0.0016
Epoch [27/100], Loss: 0.0019
Epoch [28/100], Loss: 0.0014
Epoch [29/100], Loss: 0.0012
Epoch [30/100], Loss: 0.0013
Epoch [31/100], Loss: 9.5759
Epoch [32/100], Loss: 3.2427
Epoch [33/100], Loss: 1.3735
Epoch [34/100], Loss: 1.0576
Epoch [35/100], Loss: 

In [6]:
from sklearn.metrics import accuracy_score

model.eval()  # Set model to evaluation mode
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:  # Assuming you have test_loader
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Test Accuracy: 82.21%
